<a href="https://colab.research.google.com/github/riccardo247/GPT-2-emoji/blob/main/GPT-2-emoji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introduction:**
This a notebook showing how to generate emotion classification with an all in text approach.
The model used is GPT-neo. This specific version was pretrained on the pile and on c4 (colossal dataset) for 700k steps. 
For this specific task the model was fine tuned on GoEmotion dataset.


**Example:** Given a comment or text in input the model will complete with an emotions in one of the 27 defined in GoEmotion dataset.

**Review:**
This is a comment:Succeeds in providing a disquiet world the long-dreaded completion of the Police Academy series . 

emotion is:...

**Remainder:**
Remember to switch to TPU runtime. Possibly you need to log into Colab


**Download all needd packages**


> package GPTNeo is needed with all the requirements



In [1]:
#@title Setup
%tensorflow_version 2.x
!git clone https://github.com/EleutherAI/GPTNeo
%cd GPTNeo
!pip3 install -q -r requirements.txt

Cloning into 'GPTNeo'...
remote: Enumerating objects: 3759, done.
remote: Counting objects: 100% (291/291), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 3759 (delta 172), reused 253 (delta 148), pack-reused 3468
Receiving objects: 100% (3759/3759), 1.44 MiB | 10.21 MiB/s, done.
Resolving deltas: 100% (2172/2172), done.
/content/GPTNeo
     |████████████████████████████████| 368kB 5.3MB/s 
     |████████████████████████████████| 14.4MB 230kB/s 
     |████████████████████████████████| 112kB 46.6MB/s 
     |████████████████████████████████| 3.4MB 35.7MB/s 
     |████████████████████████████████| 2.9MB 37.5MB/s 
     |████████████████████████████████| 1.5MB 29.7MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 2.2MB 32.7MB/s 
     |████████████████████████████████| 184kB 45.5MB/s 
     |████████████████████████████████| 1.0MB 32.7MB/s 
     |████████████████████████████████| 174kB 58.2MB/s 
     |██████████████████

**Gopogle authentication** could be needed to access the Google bucket storing weights

In [2]:
from google.colab import auth
auth.authenticate_user()
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'
core:
  account: riccardo.giacomelli@gmail.com

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

Choose the account you would like to use to perform operations for 
this configuration:
 [1] riccardo.giacomelli@gmail.com
 [2] Log in with a new account
Please enter your numeric choice:  1

You are logged in as: [riccardo.giacomelli@gmail.com].

Pick cloud proje

**Configuration**


> Remember to change running time to TPU



> The following are configurations files for the model and data. They do not need to be modified


> It is using GPT2 tokenizer





In [17]:
%%writefile configs/GPT3_XL_emotion.json

{
    "n_head": 16,
    "n_vocab": 50257,
    "embed_dropout": 0,
    "lr": 0.0002,
    "lr_decay": "cosine",
    "warmup_steps": 3000,
    "beta1": 0.9,
    "beta2": 0.95,
    "epsilon": 1e-8,
    "opt_name": "adam",
    "weight_decay": 0,
    "train_batch_size": 1,
    "attn_dropout": 0,
    "train_steps": 734136,
    "lr_decay_end" : 300000,
    "eval_steps": 20,
    "predict_steps": 0,
    "res_dropout": 0,
    "eval_batch_size": 1,
    "predict_batch_size": 1,
    "iterations": 10,
    "n_embd": 2048,
    "datasets": [["goemotions_testpred", null, null, null]], 
    "model_path": "gs://bigbird-freefly/neo_gpt3/gpt3_XL/the-eye.eu/public/AI/gptneo-release/GPT3_XL_goemotions_2ok_3steps",
    "n_ctx": 2048,
    "n_layer": 24,
    "scale_by_depth": true,
    "scale_by_in": false,
    "attention_types" :  [[["global", "local"],12]],
    "mesh_shape": "x:2,y:4",
    "layout" : "batch:x,memory_length:y,embd:y",
    "activation_function": "gelu",
    "recompute_grad": true,
    "gradient_clipping": 1.0,
    "tokens_per_mb_per_replica": 2048,
    "precision": "bfloat16",
    "padding_id" : 50257,
    "eos_id" : 50256
}

Overwriting configs/GPT3_XL_emotion.json


In [18]:
%%writefile configs/dataset_configs/goemotions_testpred.json
{
    "n_vocab": 50257,
    "path": "gs://bigbird-freefly/goemotions/test_labels_pred//goemotions*.tfrecords", 
    "eval_path": "gs://bigbird-freefly/goemotions/train_labels_exneutral//goemotions*.tfrecords",  
    "tokenizer_is_pretrained": true,
    "tokenizer_path": "gpt2",
    "eos_id" : 50256,
    "padding_id": 50257
}

Overwriting configs/dataset_configs/goemotions_testpred.json


In [5]:
!gsutil cp gs://bigbird-freefly/sst3/files/sample.py ./

Copying gs://bigbird-freefly/sst3/files/sample.py...
- [1 files][  9.1 KiB/  9.1 KiB]                                                
Operation completed over 1 objects/9.1 KiB.                                      


**Review examples**


> The following code cell writes to a txt a review example.  It provides emotion labelling. The text should be formatted in 2 lines as:


> This is the review:
...put the text of the review in this line...

emotion is:





In [6]:
%%writefile comment-00001.txt
This is a comment:Also feeling very inspired....i will definitely try doing the same thing
emotion is:

Writing comment-00001.txt


In [19]:
%%writefile comment-00002.txt
This is a comment:I've never been this sad in my life!
emotion is:

Overwriting comment-00002.txt


In [8]:
%%writefile comment-00003.txt
This is a review:Ehhh, it's an opinion, it's not wrong or right, just highly unpopular and ill-informed.
emotion is:

Writing comment-00003.txt


**Sentimen prediction:**
The following code is running prediction for the txt file and output the result to the cell. Change the filename or create a new one

>The first time it will take 10 minutes because it has to download the model weights. Second time will be 1-2 minutes. This notebook is just to show how it works but it is very slow!
Output will be the input ending in sentiment like this:

This is a comment:Your 'brother' had texted alot of girls and ended up on in Internet many times then, what a celebrity! Did everyone clap as well?

emotion is: **admiration**




In [24]:
review_file ='comment-00002.txt'
!python3 main.py --model GPT3_XL_emotion --steps_per_checkpoint 100 --tpu colab --predict --prompt $review_file

2021-06-05 01:15:25.418459: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
Current step 706748
Saving config to gs://bigbird-freefly/neo_gpt3/gpt3_XL/the-eye.eu/public/AI/gptneo-release/GPT3_XL_goemotions_2ok_3steps
2021-06-05 01:15:34.193401: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-06-05 01:15:34.204630: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-06-05 01:15:34.204691: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b0f6914c9aec): /proc/driver/nvidia/version does not exist
Done!
params = defaultdict(<function fetch_model_params.<locals>.<lambda> at 0x7feab4cbc290>, {'n_head': 16, 'n_vocab': 50

**Run full test data prediction**

> All the test set saved in a tfrecord file gs://bigbird-freefly/goemotions/test_labels_pred//


>Prediction result will be saved in /test_pred/test_pred_full.jsonl. Running time is about 40 minutes



> Som parameters are still hard-coded in the code. For example sampling temperature is set to 0. I overwrite few files here before execution













In [23]:
!mkdir test_pred
!gsutil cp gs://bigbird-freefly/neo_gpt3/gpt3_XL/the-eye.eu/public/AI/gptneo-release/GPT3_XL_goemotions_2ok_3steps/main_predict.py ./
!gsutil cp gs://bigbird-freefly/neo_gpt3/gpt3_XL/the-eye.eu/public/AI/gptneo-release/GPT3_XL_goemotions_2ok_3steps/inputs.py ./

mkdir: cannot create directory ‘test_pred’: File exists
Copying gs://bigbird-freefly/neo_gpt3/gpt3_XL/the-eye.eu/public/AI/gptneo-release/GPT3_XL_goemotions_2ok_3steps/main_predict.py...
/ [1 files][ 11.8 KiB/ 11.8 KiB]                                                
Operation completed over 1 objects/11.8 KiB.                                     
Copying gs://bigbird-freefly/neo_gpt3/gpt3_XL/the-eye.eu/public/AI/gptneo-release/GPT3_XL_goemotions_2ok_3steps/inputs.py...
/ [1 files][ 23.0 KiB/ 23.0 KiB]                                                
Operation completed over 1 objects/23.0 KiB.                                     


In [ ]:
%%writefile configs/GPT3_XL_emotion.json

{
    "n_head": 16,
    "n_vocab": 50257,
    "embed_dropout": 0,
    "lr": 0.0002,
    "lr_decay": "cosine",
    "warmup_steps": 3000,
    "beta1": 0.9,
    "beta2": 0.95,
    "epsilon": 1e-8,
    "opt_name": "adam",
    "weight_decay": 0,
    "train_batch_size": 8,
    "attn_dropout": 0,
    "train_steps": 734136,
    "lr_decay_end" : 300000,
    "eval_steps": 20,
    "predict_steps": 0,
    "res_dropout": 0,
    "eval_batch_size": 8,
    "predict_batch_size": 8,
    "iterations": 10,
    "n_embd": 2048,
    "datasets": [["goemotions_testpred", null, null, null]], 
    "model_path": "gs://bigbird-freefly/neo_gpt3/gpt3_XL/the-eye.eu/public/AI/gptneo-release/GPT3_XL_goemotion_2ok_3steps",
    "n_ctx": 2048,
    "n_layer": 24,
    "scale_by_depth": true,
    "scale_by_in": false,
    "attention_types" :  [[["global", "local"],12]],
    "mesh_shape": "x:2,y:4",
    "layout" : "batch:x,memory_length:y,embd:y",
    "activation_function": "gelu",
    "recompute_grad": true,
    "gradient_clipping": 1.0,
    "tokens_per_mb_per_replica": 2048,
    "precision": "bfloat16",
    "padding_id" : 50257,
    "eos_id" : 50256
}

Overwriting configs/GPT3_XL_bakeoff.json


In [ ]:
!python3 main_predict.py --predict --steps_per_checkpoint 200 --tpu colab --model GPT3_XL_emotion

In [ ]:
!head ./test_pred/test_pred.jsonl